# 自定义层
深度学习成功背后的一个因素是神经网络的灵活性： 我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。 在一些情况下，所需的层在深度学习框架中并不存在，这时就需要自定义层。本节将介绍如何构建自定义层。
## 不带参数的层
首先我们构造一个没有任何参数的自定义层`CenteredLayer`，它要从其输入减去均值。要构建它，我们只需继承基础层类并实现前向传播功能。

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        X = X - X.mean()
        return X

In [28]:
X = torch.normal(0, 0.1, size=(4, 4))

In [33]:
layer = CenteredLayer()

In [34]:
layer(X)

tensor([[ 0.0224, -0.0551,  0.0648, -0.0272],
        [-0.0908,  0.0208, -0.1151,  0.0659],
        [-0.0240,  0.0608,  0.1003,  0.0536],
        [ 0.0923, -0.0265, -0.0500, -0.0923]])

现在，我们可以将该层作为组件合并到更复杂的模型中。

In [36]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数

In [38]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.6043e-10, grad_fn=<MeanBackward0>)

## 带参数的层
以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。

In [40]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化MyLinear类并访问其模型参数。

In [42]:
linear = MyLinear(5, 3)
linear.weight.data, linear.bias.data

(tensor([[-1.3153, -0.1180,  0.7370],
         [ 1.4944,  0.3203,  0.8976],
         [-1.2800, -1.4962, -0.0744],
         [-0.9668, -1.6343, -0.9816],
         [ 1.0671,  0.6967, -0.9777]]),
 tensor([-1.4727, -0.4079,  0.1799]))

我们也可以使用自定义层直接执行前向传播计算：

In [44]:
X = torch.randn(10, 5)
linear(X)

tensor([[0.0000, 3.5314, 0.0000],
        [0.2009, 0.0447, 0.0000],
        [0.5489, 0.0000, 2.1578],
        [1.8162, 2.3333, 0.5779],
        [0.0000, 0.0000, 0.9124],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6048],
        [0.7073, 2.0737, 1.6680],
        [0.0000, 0.0000, 0.0000],
        [0.3493, 0.8240, 2.3748]])

我们还可以(使用自定义层构建模型)，就像使用内置的全连接层一样使用自定义层。

In [46]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.randn(2, 64))

tensor([[2.4212],
        [0.0000]])

## 小结
* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。